In [1]:
#creazione di un excel con colonne utii per definire i timestep
import pandas as pd

# Definizione delle stagioni e generazione dei giorni
seasons = ['winter', 'mid-cold', 'mid-warm', 'summer']
season_codes = ['wi', 'mc', 'mw', 'su']

days = []
for season, code in zip(seasons, season_codes):
    for i in range(1, 11):
        day_number = f"{i:02}"
        days.append((f"{season}{day_number}", code, day_number))

# Definizione delle ore
hours = [f"h{i:02}" for i in range(1, 25)]

# Generazione del file Excel

# Crea una lista di righe, ciascuna con l'ora e il giorno corrispondente
data = []
for day, code, num in days:
    for hour in hours:
        data.append([f"{hour}-{code}{num}", day])

# Crea un DataFrame da queste righe
df = pd.DataFrame(data, columns=['Hour', 'Day'])

filename = 'days_hours_new.xlsx'
# Scrive il DataFrame in un file Excel
df.to_excel(filename, index=False)

In [ ]:
#Copia di dati pre-esistenti ??? non provato

import pandas as pd

def fill_excel(file1, file2, output_file='output.xlsx'):
    # Carica i due file Excel
    xls1 = pd.ExcelFile(file1)
    xls2 = pd.ExcelFile(file2)

    # Dizionario per memorizzare i dati dei fogli
    sheet_data = {}

    # Elabora ogni foglio che esiste in entrambi i file
    for sheet_name in xls1.sheet_names:
        if sheet_name in xls2.sheet_names:
            # Leggi i dati del foglio da entrambi i file
            df1 = pd.read_excel(xls1, sheet_name=sheet_name)
            df2 = pd.read_excel(xls2, sheet_name=sheet_name)

            # Escludi l'ultima colonna per il confronto delle stringhe
            df1_strings = df1.iloc[:, :-1]
            df2_strings = df2.iloc[:, :-1]

            # Crea una nuova colonna per i valori da copiare
            new_column = []

            # Per ogni riga in df2, cerca il valore corrispondente in df1
            for idx, row in df2_strings.iterrows():
                match = df1_strings.apply(lambda x: (x == row).all(), axis=1)
                if match.any():
                    value = df1.iloc[match.idxmax(), -1]  # Prendi il valore dalla colonna più a destra di df1
                    new_column.append(value)
                else:
                    new_column.append(None)  # Se non c'è corrispondenza, lascia vuoto

            # Aggiungi la nuova colonna a df2
            df2.iloc[:, -1] = new_column

            # Salva i dati elaborati nel dizionario
            sheet_data[sheet_name] = df2

    # Scrivi i dati nel file di output
    with pd.ExcelWriter(output_file) as writer:
        for sheet_name, df in sheet_data.items():
            df.to_excel(writer, sheet_name=sheet_name, index=False)

# Esempio di utilizzo
file1 = 'file1.xlsx'  # File pieno
file2 = 'file2.xlsx'  # File da riempire

fill_excel(file1, file2)